In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
import matplotlib.pyplot as plt
from DateTime import DateTime

Read text data from file and set one hot encoding for product category and part of day of transactions.

In [18]:
file = pd.read_csv('train_ds.csv',sep = ',')
temp_time = file['trandatetime'].apply(lambda x: 'morning' if DateTime(x).parts()[3]<9 else ('before_dinner' if DateTime(x).parts()[3]<12 else('dinner' if DateTime(x).parts()[3]<15 else('after_dinner' if DateTime(x).parts()[3]<18 else 'evening'))))
file['trandatetime'] = temp_time
one_hot_file = pd.get_dummies(file,prefix=['mcc','time'], columns=['mcc','trandatetime']).drop(['location','cgsettlementbufferid','sexid','tranccy','ccy'],axis=1)
one_hot_file = one_hot_file.groupby(['clientid']).sum()
y_res = file.groupby(['clientid'])['sexid'].mean()
separator =  int((one_hot_file.shape[1]/10)*8)
train_model = one_hot_file[:separator]
test_model = one_hot_file[separator:]

In [19]:
rf = RandomForestClassifier(n_estimators = 15, random_state = 0,max_depth=100)
rf.fit(one_hot_file,y_res)
#print('Result random forest: {:.3f}'.format(rf.score(one_hot_file[separator:],y_res[separator:])))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=15, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [33]:
file_test = pd.read_csv('test_ds.csv',sep = ',').drop(['location','cgsettlementbufferid','tranccy','ccy'],axis=1)
temp_time = file_test['trandatetime'].apply(lambda x: 'morning' if DateTime(x).parts()[3]<9 else ('before_dinner' if DateTime(x).parts()[3]<12 else('dinner' if DateTime(x).parts()[3]<15 else('after_dinner' if DateTime(x).parts()[3]<18 else 'evening'))))
file_test['trandatetime'] = temp_time
one_hot_file_test = pd.get_dummies(file_test,prefix=['mcc','time'], columns=['mcc','trandatetime'])
one_hot_file_test = one_hot_file_test.groupby(['clientid']).sum()

In [34]:
for i in one_hot_file.head():
    if i not in one_hot_file_test.head():
        one_hot_file_test[i]=pd.Series(np.zeros(len(file_test)))
one_hot_file_test = one_hot_file_test.fillna(0)

In [38]:
arr_to_drop = []
for i in one_hot_file_test.head():
    if i not in one_hot_file.head():
        arr_to_drop.append(i)
one_hot_file_test = one_hot_file_test.drop(arr_to_drop,axis=1)

In [48]:
result = rf.predict(one_hot_file_test)
res = pd.DataFrame({'clientid':one_hot_file_test.index.values, 'sexid' : result})
res.to_csv('result_task1.csv',sep=',')

Not ready yet, by in process. Trying to group by client, type of transaction and part of day, when it was done. Then create new dataframe with amount of money spent on this type of transaction and part of day. That would be new faetures to use it on Random Forest classifier.

In [ ]:
grouped_mcc = file.groupby(['clientid','mcc','trandatetime'])['amount'].sum()
clients = pd.DataFrame(file.groupby(['clientid'])['clientid'].mean())
array_types = []
for i in clients['clientid']:
    for j in grouped_mcc[i].keys():
        temp = str(j[0])+'_'+j[1]
        if temp not in array_types:
            array_types.append(temp)
            clients[temp] = pd.Series(np.zeros(len(clients['clientid'])),index=clients.index)
            clients.loc[clients['clientid']==i][temp] = grouped_mcc[i][j[0]][j[1]]
        else:
            clients.loc[clients['clientid']==i][temp] = grouped_mcc[i][j[0]][j[1]]
clients

In [ ]:
file_test = pd.read_csv('test_ds.csv',sep = ',')drop(['location','cgsettlementbufferid','tranccy','ccy'],axis=1)
grouped_mcc_test = file_test.groupby(['clientid','mcc','trandatetime'])['amount'].sum()
clients_test = pd.DataFrame(file_test.groupby(['clientid'])['clientid'].mean())
count = len(clients_test['clientid'])
array_types = []
for i in clients_test['clientid']:
    for j in grouped_mcc_test[i].keys():
        temp = str(j[0])+'_'+j[1]
        if temp not in array_types:
            array_types.append(temp)
            clients_test[temp] = pd.Series(np.zeros(count,index=clients_test.index)
            clients_test.loc[clients_test['clientid']==i][temp] = grouped_mcc_test[i][j[0]][j[1]]
        else:
            clients_test.loc[clients_test['clientid']==i][temp] = grouped_mcc_test[i][j[0]][j[1]]

Second task

In [25]:
x = DateTime('2015/7/6 00:01am EET').timeTime()
y = DateTime('2015/7/6 11:59pm EET').timeTime()
june = DateTime('2015/6/1 00:01pm EET').timeTime()
june_end = DateTime('2015/6/30 11:59pm EET').timeTime()
start = DateTime('2015/7/1 00:01am EET').timeTime()
file = pd.read_csv('train_ds.csv',sep = ',')
file_val = file.values
fl = 0 
users = []
for i in file_val:
    if i[6]>x and i[6]<y and DateTime(i[6]).parts()[3]>17 and ('KIYEV' in i[5] or 'KYIV' in i[5] or 'KUIV' in i[5]) and i[1]==5812:
        print(DateTime(i[6]).parts(),i[8])
        fl = 0
        print('===============================')
        for j in file.loc[file['clientid']==i[8]].values:
            if j[6]<start:
                print(DateTime(j[6]).parts(),j[5])
            if j[6]<start and (j[1]==4511 or (j[1]>=3000 and j[1]<=3299)):
                #print(DateTime(j[6]).parts(), j[8], j[5],j[4],j[3],j[2])
                fl = 1
        if fl == 1:
            for j in file.loc[file['clientid']==i[8]].values:
                if 'UA' in j[5] and j[6]>june and j[6]<june_end:
                    #print(DateTime(j[6]).parts(), j[8], j[5],j[4],j[3],j[2])
                    fl=0
            if fl==1:
                users.append(i[8])
        
        print('+++++++++++++++++++++++++++++++')
print(users)
print("John Dow's id: 2164 and he bought tikets in WIZZ AIR HUN920WZZP9KVJ BUDAPEST HU, so he lives in Hungary Budapest")

(2015, 7, 6, 22, 57, 59, 'GMT+3') 5273
(2015, 3, 6, 16, 32, 23, 'GMT+2') BOOKING UZ             KIEV           UA
(2015, 3, 11, 7, 13, 11, 'GMT+2') WWW.SIMMARKET.COM      MAINZ          DE
(2015, 4, 25, 15, 2, 3, 'GMT+3') RAILWAYTICKETS         KYIV           UA
(2015, 4, 25, 15, 9, 37, 'GMT+3') RAILWAY_TICK           KYIV           UA
(2015, 6, 11, 1, 0, 0, 'GMT+3') RAILWAYTICKETS         KYIV           UA
+++++++++++++++++++++++++++++++
(2015, 7, 6, 19, 27, 53, 'GMT+3') 2164
(2014, 7, 8, 14, 17, 33, 'GMT+3') MINIMARKET24 GU        KIYEV          UA
(2014, 6, 24, 20, 9, 29, 'GMT+3') UNIVERSAM SILPO        KIYEV          UA
(2014, 6, 25, 15, 45, 28, 'GMT+3') BIRJA BAR 52           KIYEV          UA
(2014, 7, 11, 0, 32, 37, 'GMT+3') UNIVERSAM SILPO        KIYEV          UA
(2014, 6, 28, 15, 19, 49, 'GMT+3') AZS 35 OKKO            KYIV           UA
(2014, 6, 24, 19, 46, 54, 'GMT+3') AZS 27                 KYIV           UA
(2014, 6, 29, 18, 32, 40, 'GMT+3') 011_SUSHIYA            KIYEV  

(2015, 2, 7, 15, 5, 47, 'GMT+2') VELOBAZAR27            KIYEV          UA
(2015, 2, 7, 15, 45, 38, 'GMT+2') SANDVICHBAR            KYIV           UA
(2015, 2, 8, 16, 41, 32, 'GMT+2') WWW.WIZZAIR.COM        KIEV           UA
(2015, 2, 15, 3, 40, 16, 'GMT+2') PORTMONE MOBILE        KYIV           UA
(2015, 2, 15, 22, 21, 32, 'GMT+2') AVTO ZAPRAVNYI KOMPLEKSKYIV           UA
(2015, 2, 18, 15, 4, 54, 'GMT+2') KIEVSKOE RU            KYIV           UA
(2015, 2, 19, 19, 51, 35, 'GMT+2') UNIVERSAMSILPO00002470 KIYEV          UA
(2015, 2, 22, 0, 23, 11, 'GMT+2') KREDOBANK              KYIV           UA
(2015, 2, 21, 22, 45, 12, 'GMT+2') McDonald's 063         KYIV           UA
(2015, 2, 23, 19, 20, 6, 'GMT+2') MULTIPLEX              KYIV           UA
(2015, 2, 23, 21, 40, 59, 'GMT+2') AZS 27                 KYIV           UA
(2015, 2, 23, 22, 15, 49, 'GMT+2') UNIVERSAMSILPO00002470 KIYEV          UA
(2015, 2, 25, 9, 5, 47, 'GMT+2') UNIVERSAMSILPO00002470 KIYEV          UA
(2015, 2, 24, 19, 55, 

(2015, 6, 20, 20, 59, 31, 'GMT+3') FOPVAVRINEVICH         KIYEV          UA
(2015, 6, 21, 8, 38, 55, 'GMT+3') DUTY FREE              BORYSPIL       UA
+++++++++++++++++++++++++++++++
(2015, 7, 6, 22, 46, 51, 'GMT+3') 5293
(2014, 12, 15, 11, 53, 22, 'GMT+2') KYIV OFFICE 2          KYIV           UA
(2014, 12, 16, 20, 53, 27, 'GMT+2') KYIV OFFICE 2          KYIV           UA
(2014, 12, 16, 13, 58, 32, 'GMT+2') AZK WOG CHERVONOZORIANYKYIV           UA
(2014, 12, 18, 10, 30, 59, 'GMT+2') KYIV OFFICE 2          KYIV           UA
(2014, 12, 25, 14, 58, 32, 'GMT+2') PAT "BANK SICH/SHYTOVA KIEV           UA
(2014, 12, 25, 9, 30, 34, 'GMT+2') AZS WOG BORYSPILSKA 12 KYIV           UA
(2014, 12, 26, 11, 16, 59, 'GMT+2') KYIV OFFICE 2          KYIV           UA
(2014, 12, 31, 15, 13, 55, 'GMT+2') KYIV OFFICE 2          KYIV           UA
(2015, 1, 6, 16, 31, 47, 'GMT+2') KYIV OFFICE 2          KYIV           UA
(2015, 1, 8, 13, 18, 41, 'GMT+2') KYIV OFFICE 2          KYIV           UA
(2015, 1, 16,

(2015, 4, 10, 23, 37, 52, 'GMT+3') RESTORAN OLD CONTINENT UZHGOROD     UAUA
(2015, 4, 11, 13, 26, 18, 'GMT+3') PARFUMSTUDIOUZH        UZHGOROD     UAUA
(2015, 4, 11, 14, 4, 42, 'GMT+3') KOBZAR                 UZHGOROD     UAUA
(2015, 4, 11, 14, 9, 19, 'GMT+3') KOBZAR                 UZHGOROD     UAUA
(2015, 4, 15, 11, 0, 29, 'GMT+3') SHOP GOODWINE          KYIV         UAUA
(2015, 4, 15, 15, 12, 43, 'GMT+3') Kornercontce39         KIYEV        UAUA
(2015, 4, 16, 19, 37, 41, 'GMT+3') MTICKET                KIEV         UAUA
(2015, 4, 15, 16, 10, 56, 'GMT+3') BK CARAMEL             KYIV         UAUA
(2015, 4, 15, 17, 41, 14, 'GMT+3') SHOP GOODWINE          KYIV         UAUA
(2015, 4, 18, 19, 25, 30, 'GMT+3') MAGAZIN3000253         KIYEV        UAUA
(2015, 4, 17, 18, 13, 59, 'GMT+3') BAMBINI FELICI         KYIV         UAUA
(2015, 4, 17, 18, 40, 3, 'GMT+3') 22TASAPTEKA            KIYEV        UAUA
(2015, 4, 19, 14, 16, 51, 'GMT+3') SHOP NEW LIFE          KYIV         UAUA
(2015, 4, 20, 12

(2015, 6, 2, 22, 36, 1, 'GMT+3') SHOP FURSHET VUSHGOROD KYIV         UAUA
(2015, 6, 3, 11, 38, 50, 'GMT+3') OLDI                   KIEV           UA
(2015, 6, 2, 22, 31, 9, 'GMT+3') SHOP FURSHET VUSHGOROD KYIV         UAUA
(2015, 6, 3, 20, 12, 22, 'GMT+3') ZBB19320649            KIYEV        UAUA
(2015, 6, 5, 21, 13, 14, 'GMT+3') DOMINOSPICCAYuKREYN    KIYEV        UAUA
(2015, 6, 6, 21, 58, 9, 'GMT+3') UNIVERSAMSILPO00002476 KIYEV        UAUA
(2015, 6, 7, 12, 15, 54, 'GMT+3') CARAVAN                KYIV         UAUA
(2015, 6, 8, 14, 14, 56, 'GMT+3') BELORUSSKAYA2          KIYEV        UAUA
(2015, 6, 9, 9, 43, 30, 'GMT+3') McDonald's 032         KYIV         UAUA
(2015, 6, 10, 15, 7, 13, 'GMT+3') MFPay2youAC            Kiiev        UAUA
(2015, 6, 11, 15, 11, 19, 'GMT+3') WWW.PORTMONE.COM.UA.TELKYIV           UA
(2015, 6, 12, 14, 18, 54, 'GMT+3') KYIV RU                KYIV           UA
(2015, 6, 11, 14, 28, 26, 'GMT+3') SAMSONE6               KIYEV        UAUA
(2015, 6, 12, 16, 46, 41, 